## Load Inpainting Model

In [1]:
import torch
from diffusers import AutoPipelineForInpainting
from diffusers.utils import load_image, make_image_grid

pipeline = AutoPipelineForInpainting.from_pretrained(
    "runwayml/stable-diffusion-inpainting", torch_dtype=torch.float16, variant="fp16"
)
pipeline.enable_model_cpu_offload()

/home/yangyo/anaconda3/envs/DL/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/yangyo/anaconda3/envs/DL/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Loading pipeline components...: 100%|██████████| 7/7 [00:00<00:00,  9.85it/s]


In [15]:
import os
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from tqdm import tqdm
import shutil

In [16]:
os.getcwd()

'/home/yangyo/Desktop/DeepLearning'

In [17]:
imgs_path = os.getcwd() + "/test20_image/"
labels_path = os.getcwd() + "/test20_label/"
pos_path = os.getcwd() + "/test20_pos/"
save_path = os.getcwd() + "/stable_diffusion_inpainting/"

## Caculate Object coordinate

In [19]:
labels = os.listdir(labels_path)
for label in labels:
    if label != ".ipynb_checkpoints":
        with open(labels_path + label, 'r') as read_file, open(pos_path + label, 'w') as write_file:
            lines = read_file.readlines()
            for i in range(1, len(lines)):
                output = lines[i].split(' ')
                w = float(output[2]) * 1024.0  #object width
                h = float(output[3]) * 1024.0  #object height
                x = int(float(output[0]) * 1024.0 - (w / 2)) # object Xmin
                y = int(float(output[1]) * 1024.0 - (h / 2)) # object Ymin
                write_file.write(f"{x} {y} {int(w)} {int(h)} {output[4]}") # save as .txt

## Create Mask Image

In [20]:
def create_mask(coordinate_path, conf):
    arr = np.zeros((1024, 1024, 3))
    with open(coordinate_path, 'r') as f:
        for line in f.readlines():
            info = line.split(' ')
            if float(info[4]) > conf:
              arr[int(info[1]): int(info[1]) + int(info[3]), int(info[0]): int(info[0]) + int(info[2]),  :] = 255
    return arr

## Draw Original Image & Mask Image & Inpainting

In [21]:
prompt = "Clear, without acne , nature,  skin"
negative_prompt = "bad anatomy, deformed, ugly, disfigured"
generator = torch.Generator("cuda").manual_seed(92)

def inpaint(image_path, pos_path, label, conf, prompt, negative_prompt, save_path, generator):
    number = label[:-4]
    img = Image.open(image_path + number + ".jpg")
    mask = create_mask(pos_path + number + ".txt", conf)
    mask_img = Image.fromarray(mask.astype(np.uint8))
    prompt = prompt
    negative_prompt = negative_prompt
    image = pipeline(prompt=prompt, negative_prompt=negative_prompt, image=img, mask_image=mask_img, generator=generator).images[0]
    if save_path != None:
        image.save(save_path + f"{number}.jpg")
    fig, axes = plt.subplots(nrows=1, ncols=3, figsize=(8, 6))
    axes[0].set_title(f"levle1_{number}.jpg")
    axes[1].set_title(f"Mask")
    axes[2].set_title(f"Inpainting")
    axes[0].imshow(img)
    axes[1].imshow(mask_img)
    axes[2].imshow(image)

## Test All

In [23]:
for label in labels:
    inpaint(imgs_path , pos_path , label, 0.0, prompt, negative_prompt, save_path, generator)

100%|██████████| 50/50 [00:05<00:00,  9.58it/s]
Potential NSFW content was detected in one or more images. A black image will be returned instead. Try again with a different prompt and/or seed.
